In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [107]:
import kagglehub
import os
# Download latest version
folder_path = kagglehub.dataset_download("cheedcheed/top1m")

print("Path to dataset files:", folder_path)
csv_files = [f for f in os.listdir(folder_path) if f.lower().endswith('.csv')]

if not csv_files:
    raise FileNotFoundError("No CSV files found in the folder!")

# read the first CSV file
file_path = os.path.join(folder_path, csv_files[0])
alexa_top_1m_domain = pd.read_csv(file_path,header=None,names=['rank', 'domain'])
alexa_domains_set = set(alexa_top_1m_domain['domain'].apply(str.lower))

Path to dataset files: C:\Users\rrpra\.cache\kagglehub\datasets\cheedcheed\top1m\versions\1


In [73]:
from datasets import load_dataset
import pandas as pd

# Load the dataset from Hugging Face Hub
train_dataset = load_dataset("kmack/Phishing_urls", split="train")
test_dataset = load_dataset("kmack/Phishing_urls", split="test")
valid_dataset = load_dataset("kmack/Phishing_urls", split="valid")

# Convert to pandas DataFrame
train_df = train_dataset.to_pandas()
test_df = test_dataset.to_pandas()
valid_df = valid_dataset.to_pandas()

print(train_df.head())


                                                text  label
0             xenophongroup.com/montjoie/compgns.htm      0
1    www.azzali.eu/&usg=AOvVaw2phVSb_ENMrkATGNx5LQ0l      1
2                     guildmusic.edu.au/js/index.htm      1
3  memo.unexpectedrunner.com/ezxgytw4et\nholotili...      1
4  en.wikipedia.org/wiki/Category:American_televi...      0


In [74]:
import tldextract
df = pd.concat((train_df, valid_df, test_df))
def extrtact_tld(url):

    ext = tldextract.extract(url)
    # ext = ExtractResult(subdomain, domain, suffix)
    tld = ext.suffix    # "com", "co.uk", "org"
    return tld if tld else 'unknown'

df['tld'] = df['text'].apply(extrtact_tld)
df.head()

,text,label,tld
0,xenophongroup.com/montjoie/compgns.htm,0,com
1,www.azzali.eu/&usg=AOvVaw2phVSb_ENMrkATGNx5LQ0l,1,eu
2,guildmusic.edu.au/js/index.htm,1,edu.au
3,memo.unexpectedrunner.com/ezxgytw4et\nholotili...,1,com
4,en.wikipedia.org/wiki/Category:American_televi...,0,org


In [75]:
# Count phishing and legitimate occurrences for each TLD
tld_stats = df.groupby(['tld', 'label']).size().unstack(fill_value=0)
tld_stats.columns = ['legit_count', 'phish_count']

# Add totals and phishing rate
tld_stats['total'] = tld_stats['legit_count'] + tld_stats['phish_count']
tld_stats['phish_ratio'] = tld_stats['phish_count'] / tld_stats['total']
tld_stats = tld_stats.sort_values('phish_ratio', ascending=False)

print(tld_stats)

                    legit_count  phish_count  total  phish_ratio
tld                                                             
Tokyo                         0            1      1          1.0
go.gov.br                     0            3      3          1.0
goog                          0           13     13          1.0
gop.pk                        0            1      1          1.0
gov.la                        0            3      3          1.0
...                         ...          ...    ...          ...
kagoshima.jp                  1            0      1          0.0
kanagawa.jp                   1            0      1          0.0
kasaoka.okayama.jp            1            0      1          0.0
katowice.pl                   1            0      1          0.0
ye                            1            0      1          0.0

[1437 rows x 4 columns]


In [134]:
!pip install pybktree


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pybktree: filename=pybktree-1.1-py3-none-any.whl size=5042 sha256=2d3cf7c2702be22561b85ce73a053cd7caedb17e0bcc7575f4e6bb62e51acf46
  Stored in directory: c:\users\rrpra\appdata\local\pip\cache\wheels\d9\f0\dd\545edc6df9148221657c15a0c0a42ec00b216c8c9d42d559e9
Successfully built pybktree


  DEPRECATION: Building 'pybktree' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'pybktree'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [138]:
import pandas as pd
import re
import math
from urllib.parse import urlparse
import tldextract
from rapidfuzz import process, fuzz
# --- Helper function: Shannon entropy ---
def safe_parse(url: str):
    """Safely parse URLs, adding http:// if missing and handling bad IPv6 parts."""
    if not isinstance(url, str) or not url.strip():
        return urlparse("http://")  

    # Ensure scheme exists
    if not re.match(r'^[a-zA-Z]+://', url):
        url = 'http://' + url

    # Clean invalid brackets that trigger IPv6 errors
    url = re.sub(r'\[.*?\]', '', url)

    try:
        return urlparse(url)
    except ValueError:
        # fallback: strip more aggressively if still malformed
        url = re.sub(r'[^a-zA-Z0-9:/._\-?&=]', '', url)
        return urlparse(url)
def calculate_entropy(string):
    """Measures randomness of characters in the URL."""
    if not string:
        return 0
    freq = {char: string.count(char) for char in set(string)}
    entropy = -sum((count / len(string)) * math.log2(count / len(string)) for count in freq.values())
    return entropy

# --- Main feature extraction function ---
def extract_features(url):
    features = {}
    if not re.match(r'^[hH]+[tT]+[tT]+[pP]+[sS]+://', url):
        url = 'http://' + url
    parsed = safe_parse(url)
    
    # 1️⃣ Basic structural features
    features['url_length'] = len(url)
    features['hostname_length'] = len(parsed.netloc)
    features['path_length'] = len(parsed.path)
    features['num_dots'] = url.count('.')
    features['num_hyphens'] = url.count('-')
    features['num_digits'] = sum(c.isdigit() for c in url)
    features['num_letters'] = sum(c.isalpha() for c in url)
    features['num_params'] = url.count('?')
    features['num_equals'] = url.count('=')
    features['num_slashes'] = url.count('/')
    features['num_at'] = url.count('@')

    # 2️⃣ Lexical / composition cues
    features['has_https'] = 1 if url.lower().startswith('https') else 0
    features['has_ip'] = 1 if re.search(r'(\d{1,3}\.){3}\d{1,3}', parsed.netloc) else 0
    features['has_subdomain'] = 1 if parsed.netloc.count('.') > 1 else 0
    features['has_suspicious_words'] = 1 if re.search(r'(login|secure|verify|update|free|bank|click)', url.lower()) else 0

    # 3️⃣ Domain / TLD features
    extracted = tldextract.extract(url)
    main_domain = f"{extracted.domain}.{extracted.suffix}"
    if ':' in main_domain:  # remove port
        main_domain = main_domain.split(':')[0]
    features['domain_length'] = len(main_domain)
    features['in_alexa_top1m'] = 1 if main_domain in alexa_domains_set else 0
    '''
    if features['in_alexa_top1m'] == 0 and main_domain:  # only check if domain not in top1M
        # find closest match in Alexa domains
        best_match, score, _ = process.extractOne(main_domain, alexa_domains_set, scorer=fuzz.ratio)
        features['closest_alexa_domain'] = best_match
        features['closest_alexa_score'] = score  # 0-100
    else:
        features['closest_alexa_score'] = 1000  # high score to show that it is original url
    '''
    ext = tldextract.extract(url)
    tld = ext.suffix    # "com", "co.uk", "org"
    features['tld'] = tld if tld else 'unknown'
    features['tld_phish_ratio'] = tld_stats['phish_ratio'][features['tld']]  if tld_stats['phish_ratio'][features['tld']]  else 0.5
    features['tld_total_frequency'] = tld_stats['total'][features['tld']] if tld_stats['total'][features['tld']] else 1

    # 4️⃣ Ratios
    features['digit_ratio'] = features['num_digits'] / (features['url_length'] + 1e-5)
    features['special_char_ratio'] = (features['num_hyphens'] + features['num_dots'] + features['num_slashes']) / (features['url_length'] + 1e-5)

    # 5️⃣ Entropy (measures randomness / obfuscation)
    features['url_entropy'] = calculate_entropy(url)

    # 6️⃣ Misplacement indicators
    # '@' symbol used to hide real domain (like "http://evil.com@legit.com")
    features['at_in_domain'] = 1 if '@' in parsed.netloc else 0
    
    # Double slashes '//' appearing after path (used to trick users)
    features['double_slash_in_path'] = 1 if re.search(r'/.+//', parsed.path) else 0

    return features

In [129]:
for i in range(5):
    print(df.text[i])

xenophongroup.com/montjoie/compgns.htm
www.azzali.eu/&usg=AOvVaw2phVSb_ENMrkATGNx5LQ0l
guildmusic.edu.au/js/index.htm
memo.unexpectedrunner.com/ezxgytw4et\nholotilicga.medproshealthcare.com/bannerbox/frame.html
en.wikipedia.org/wiki/Category:American_television_films


In [140]:
from tqdm import tqdm
tqdm.pandas()
all_df = [train_df, test_df,valid_df]
for i, df in enumerate(all_df):
    all_df[i] = df.assign(**df.text.progress_apply(lambda url : pd.Series(extract_features(url))))
train_df, test_df,valid_df = all_df

100%|██████████| 70882/70882 [00:16<00:00, 4174.41it/s]


In [141]:
def dataframe_summary(df):
    summary = pd.DataFrame({
        "DataType": df.dtypes,
        "Non-Null Count": df.count(),
        "Missing Values": df.isnull().sum(),
        "Unique Values": df.nunique(),
        "First Value": df.iloc[0],
        "Sample Values": df.apply(lambda x: x.unique()[:5])  # first 5 unique values
    })
    return summary

# Example usage
summary_table = dataframe_summary(train_df)
print(summary_table)

                     DataType  Non-Null Count  Missing Values  Unique Values  \
text                   object          567056               0         535838   
label                   int64          567056               0              2   
tld                    object          567056               0           1341   
tld_phish_ratio       float64          567056               0            348   
tld_total_frequency     int64          567056               0            252   
url_length              int64          567056               0            704   
hostname_length         int64          567056               0            243   
path_length             int64          567056               0            405   
num_dots                int64          567056               0             37   
num_hyphens             int64          567056               0             47   
num_digits              int64          567056               0            269   
num_letters             int64          5

In [144]:
train_df.to_csv(r'Dataset\url_train_df.csv',index=False)
test_df.to_csv(r'Dataset\url_test_df.csv',index=False)
valid_df.to_csv(r'Dataset\url_validation_df.csv',index=False)